> Copyright 2022 University of Luxembourg
> 
> Licensed under the Apache License, Version 2.0 (the "License");  
> you may not use this file except in compliance with the License.  
> You may obtain a copy of the License at  
>
>    https://www.apache.org/licenses/LICENSE-2.0
>
> Unless required by applicable law or agreed to in writing, software  
> distributed under the License is distributed on an "AS IS" BASIS,  
> WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  
> See the License for the specific language governing permissions and  
> limitations under the License.  
>
***

Author: André Stemper (andre.stemper@uni.lu)

***

# Postprocessing - Check if dropit did fail to update data during transfer


Measurement results directory


In [ ]:
if not 'measurements_root' in locals():
    measurements_root = './measurements'
    

In [ ]:
import json
import sys
import os
import hashlib
%matplotlib inline


In [ ]:
checksums = {}

In [ ]:
def check_experiment(experiment_timestamp_string, d):
    if not os.path.exists(d+os.path.sep+'testbench.pdf'):
        raise Exception("Experiment {} is missing testbench.pdf".format(experiment_timestamp_string))
    try:
        md5_voltage = hashlib.md5(open(d+os.path.sep+'voltage.csv','rb').read()).hexdigest()
    except FileExistsError as e:
        raise Exception("Experiment {} missing file: {}".format(experiment_timestamp_string, e))
    try:
        md5_current = hashlib.md5(open(d+os.path.sep+'current.csv','rb').read()).hexdigest()
    except FileExistsError as e:
        raise Exception("Experiment {} missing file: {}".format(experiment_timestamp_string, e))

    if md5_voltage in checksums.keys():
        raise Exception("Collisison for voltage trace {} {}".format(checksums[md5_voltage], (experiment_timestamp_string, 'voltage')))
    if md5_current in checksums.keys():
        raise Exception("Collisison for current trace {} {}".format(checksums[md5_current], (experiment_timestamp_string, 'current')))
    
    checksums[md5_voltage] = (experiment_timestamp_string, 'voltage')
    checksums[md5_current] = (experiment_timestamp_string, 'current')

    print("Experiment {} passed.".format(experiment_timestamp_string))

Check all experiments


In [ ]:
import os
for file in os.listdir(measurements_root):
    d = os.path.join(measurements_root, file)
    if os.path.isdir(d):
        experiment_timestamp_string = file
        try:
            check_experiment(experiment_timestamp_string, d)
        except Exception as e:
            print(e)